# Running Locally 

In [15]:
# working with ollama running locally
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser

# model name to use
model_name = "deepseek-r1:8b"

# question to ask the model
question = "What is the capital of France?"

# create a chat model chat template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{question}")
    ]
)

llm = ChatOllama(model=model_name)

# creating a chain to invoke the model with the prompt
chain = prompt | llm | StrOutputParser()

result = chain.invoke(prompt.format_prompt(question=question).to_messages())
print(result)



<think>
Okay, let's see what this query is asking for. The user wants to know the capital of France.

First, I need to recall that Paris is indeed the capital city of France. It's a well-known fact and should be straightforward unless there's some trick here. But wait, maybe the user expects something more than just the name? They might want additional information about why it's significant or any historical context around its status as the capital.

The system message identifies me as a helpful assistant, so my response should be accurate and concise. The human message is pretty direct: "What is the capital of France?" No hidden layers detected here. Just need to answer correctly with Paris. 

But I should consider if the user might have deeper needs. Are they testing for basic knowledge? Maybe preparing for a test or trivia game? Or perhaps they're looking for confirmation about something they already suspect but want to be sure? Since there's no indication of complexity beyond the o